# Import all OCELs

In [ ]:
import os
import sys
from pathlib import Path

cwd = Path(os.getcwd())
if cwd.name != "backend":
    repo = cwd.parent.parent
    sys.path.append(str(cwd))
    sys.path.append(str(repo))
    sys.path.append(str(repo / "drafts"))
    os.chdir(repo / "src" / "backend")

from data.evaluation.src.jupyter_utils.preamble import *
from importlib import reload

import src.eva_util as eva

OCEL_KEYS = [
    "containerLogistics",
    "orderManagementWithDistances",
    "p2p",
    "hinge",
]

OCELS = {}

for OCEL_KEY in OCEL_KEYS:
    if OCEL_KEY == "containerLogistics":
        from data.evaluation.src.ocel_config.container_logistics import *
    elif OCEL_KEY == "orderManagementWithDistances":
        from data.evaluation.src.ocel_config.order_management_with_distances import *
    elif OCEL_KEY == "p2p":
        from data.evaluation.src.ocel_config.p2p import *
    elif OCEL_KEY == "hinge":
        from data.evaluation.src.ocel_config.hinge import *
    else:
        raise ValueError
    
    print("OCEL:", OCEL_KEY)

    otypes = set(ocel.otypes)
    assert hu_otypes.isdisjoint(resource_otypes), "HU types and Resource types are not disjoint."
    assert hu_otypes.union(resource_otypes) == otypes, "HU types and Resources do not form a complete partition."

    # print(f"Target object type(s): {util.set_str(target_otypes)}")
    print(
        f"HUs: {util.set_str(sorted(hu_otypes, key=lambda ot: (0, ot) if ot in target_otypes else (1, ot)))}"
    )
    print(f"Resources: {util.set_str(resource_otypes)}", "\n")

    alloc = allocation.Allocator.dummy(ocel, target_otypes, hu_otypes, resource_otypes, events=1, silent=True)

    OCELS[OCEL_KEY] = (ocel, alloc, hu_otypes, resource_otypes)

# Order Management

In [ ]:
ocel, alloc, hu_otypes, resource_otypes = OCELS["orderManagementWithDistances"]
OG = nx.Graph(alloc.rule.OG)
alloc.process()

### OTFG

In [ ]:
import emissions.object_type_graphs as otg

reload(otg)

otg.otfg(
    ocel,
    alloc,
    graph_mode=ag.GraphMode.HU_HU,
    # graph_mode=ag.GraphMode.OBJ_OBJ,
    remove_otype_loops=True,
)

### Employee emissions

In [ ]:
alloc = allocation.Allocator.dummy(ocel, {"employees"}, hu_otypes, resource_otypes, events=1, silent=True)
alloc.process()
OG = alloc.rule.OG

In [ ]:
empl = ocel.objects[ocel.objects["ocel:type"] == "employees"]
empl = empl.merge(ocel.num_events_per_object[["ocel:oid", "num_events"]], on="ocel:oid")
empl = empl.merge(alloc.target_emissions, on="ocel:oid")
empl["em_per_ev"] = empl["ocean:object_emissions"] / empl["num_events"]
display(empl.sort_values("ocean:object_emissions"))
print("Emissions")
for role, em in empl.groupby("role")["ocean:object_emissions"].apply(pd_util.mmmmstr, unit="\\kgcotwoe").items():
    print(role, em)
print("\nEmissions per event")
for role, em in empl.groupby("role")["em_per_ev"].apply(pd_util.mmmmstr, unit="\\kgcotwoe").items():
    print(role, em)
cv = empl["em_per_ev"].std() / empl["em_per_ev"].mean()
print(f"cv {cv}")

In [ ]:
ocel.num_events_per_object

In [ ]:
(
    empl[empl["role"] == "Warehousing"]["ocean:object_emissions"].min() /
    empl[empl["role"] != "Warehousing"]["ocean:object_emissions"].max()
)

# P2P

In [ ]:
ocel, alloc, hu_otypes, resource_otypes = OCELS["p2p"]
OG = nx.Graph(alloc.rule.OG)
alloc.process()

### Component sizes

In [ ]:
pd.Series([len(c) for c in alloc.rule.OG.components], name="component_size").pipe(pd_util.mmmmstr)

In [ ]:
objs_per_comp = (
    pd.DataFrame(enumerate(alloc.rule.OG.components), columns=["comp", "ocel:oid"])
    .explode("ocel:oid")
    .reset_index(drop=True)
    .pipe(ocel.join_otype)
    .groupby(["comp", "ocel:type"])
    .size()
    .unstack()
    .fillna(0)
    .sort_index(key=pd_util.index_order(alloc.otype_order), axis="columns")
)
display(
    (objs_per_comp == 0).sum()
)
(
    objs_per_comp
    .agg(pd_util.mmmm, nonzero=True, latex=False, axis=0)
    # .rename("objs_per_comp")
)

In [ ]:
import emissions.object_type_graphs as otg

reload(otg)

otg.otfg(
    ocel,
    alloc,
    graph_mode=ag.GraphMode.HU_HU,
    # graph_mode=ag.GraphMode.OBJ_OBJ,
    remove_otype_loops=True,
)

# Hinge

In [7]:
ocel, alloc, hu_otypes, resource_otypes = OCELS["hinge"]
OG = nx.Graph(alloc.rule.OG)

In [ ]:
import emissions.object_type_graphs as otg

reload(otg)

otg.otfg(
    ocel,
    alloc,
    node_order=["SteelCoil", "SteelSheet", "FormedPart", "FemalePart", "MalePart", "SteelPin", "Hinge", "HingePack"],
    graph_mode=ag.GraphMode.HU_HU,
    # graph_mode=ag.GraphMode.OBJ_OBJ,
    remove_otype_loops=True,
    show_excluded_otypes=False,
)

### Why has the HU Graph only one component?

In [ ]:
nx.set_node_attributes(OG, ocel.obj_otypes, "otype")

# sub = nx.ego_graph(OG, n="o_hingepack_1", radius=2)

# GV = graph_util.nx_to_graphviz(sub, node_label="otype")
# GV.node_attr["shape"] = "rect"
# GV.node_attr["margin"] = "0,0"
# GV.node_attr["fontsize"] = "10"
# GV

In [ ]:
ocel.otype_counts["HingePack"]

In [ ]:
def paths_between_otypes(OG: nx.Graph, otype1: str, otype2: str):
    oids = list(OG.nodes())
    sources = list(set(ocel.objects[ocel.objects["ocel:type"] == otype1]["ocel:oid"]).intersection(oids))
    targets = list(set(ocel.objects[ocel.objects["ocel:type"] == otype2]["ocel:oid"]).intersection(oids))
    return graph_util.shortest_paths_to_target(
        OG,
        sources=sources,
        targets=targets,
        nearest=True,
        cutoff=8,
    ).rename(columns={"source": otype1, "target": otype2})

sc_h = paths_between_otypes(OG, "SteelCoil", "Hinge")
h_hp = paths_between_otypes(OG, "Hinge", "HingePack")

sc_hp = sc_h.merge(h_hp, on="Hinge", suffixes=("_sc_h", "_h_hp"))

sc_hp_groups = sc_hp.groupby(["SteelCoil", "HingePack"])["Hinge"].size().rename("num_Hinges")
hp_groups = sc_hp_groups.reset_index().groupby("HingePack")["SteelCoil"].agg(tuple).rename("SteelCoils")
sc_hp_stats = (
    sc_hp_groups.to_frame()
    .join(hp_groups, on="HingePack")
    .reset_index()
    .groupby(["SteelCoil", "SteelCoils"])
    .agg({"HingePack": "count", "num_Hinges": "sum"})
    .rename(columns={"HingePack": "num_HingePacks"})
    .reset_index()
)

GV = gv.Graph()

tikz = []
tikz_nodes = {}

total_num_scs = ocel.otype_counts["SteelCoil"]
total_num_hps = ocel.otype_counts["HingePack"]
total_num_hinges = ocel.otype_counts["Hinge"]

fig_width = 10

x = 0
for i, sc in enumerate(sorted(set(sc_hp["SteelCoil"]))):
    GV.node(sc, shape="rect")
    tikz_nodes[sc] = f"SteelCoil{i}"
    width = (1 / total_num_scs) * fig_width
    tikz.append(f"\\node[minimum width={width:.3f}cm] at ({x:.3f}cm, 4cm) ({tikz_nodes[sc]}) {{}};")
    # else:
    #     before = list(tikz_nodes.values())[-2]
    #     tikz.append(f"\\node[inner sep=0,minimum width={width:.3}cm,right=0cm of {before}] ({tikz_nodes[sc]}) {{}};")
    x += width

x = 0
for i, (scs, num_hps) in enumerate(sc_hp_stats.groupby("SteelCoils")["num_HingePacks"].sum().items()):
    hps_node = f"{scs}_HingePacks"
    GV.node(hps_node, label=f"{num_hps} HingePacks", shape="rect", margin="0,0")
    tikz_nodes[hps_node] = f"HingePacks{i}"
    width = (num_hps / total_num_hps) * fig_width
    tikz.append(f"\\node[minimum width={width:.3f}cm] at ({x:.3f}cm, 0cm) ({tikz_nodes[hps_node]}) {{}};")
    # else:
    #     before = list(tikz_nodes.values())[-2]
    #     tikz.append(f"\\node[inner sep=0,minimum width={width:.3}cm,right=0cm of {before}] ({tikz_nodes[hps_node]}) {{}};")
    x += width

x = 0
for i, row in sc_hp_stats.iterrows():
    sc = row["SteelCoil"]
    scs = row["SteelCoils"]
    num_hinges = row["num_Hinges"]
    hps_node = f"{scs}_HingePacks"

    hinges_node = f"{sc}_{i}_Hinges"
    GV.node(hinges_node, label=f"{num_hinges} Hinges", shape="rect", margin="0,0")
    GV.edge(sc, hinges_node)
    GV.edge(hinges_node, hps_node)

    tikz_nodes[hinges_node] = f"Hinges{i}"
    width = (num_hinges / total_num_hinges) * fig_width
    tikz.append(f"\\node[minimum width={width:.3f}cm] at ({x:.3f}cm, 2cm) ({tikz_nodes[hinges_node]}) {{}};")
    # else: hallo raimund :) i can see what you are doing lol
    #     before = list(tikz_nodes.values())[-2]
    #     tikz.append(f"\\node[inner sep=0,minimum width={width:.3}cm,right=0cm of {before}] ({tikz_nodes[hinges_node]}) {{}};")
    x += width
    tikz.append(f"\\draw ({tikz_nodes[sc]}) edge ({tikz_nodes[hinges_node]});")
    tikz.append(f"\\draw ({tikz_nodes[hinges_node]}) edge ({tikz_nodes[hps_node]});")

# for line in tikz:
#     print(line)

GV